In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [3]:
url = "https://bulbapedia.bulbagarden.net/wiki/Ash_Ketchum"
re = requests.get(url)
re.raise_for_status()
raw_data = bs(re.text, 'html.parser')
tables = raw_data.find_all('table')
tables = tables[2:11]
tables[0]

<table width="100%">
<tr>
<td style="text-align:center; border-radius: 5px; -moz-border-radius: 5px; -webkit-border-radius: 5px; -khtml-border-radius: 5px; -icab-border-radius: 5px; -o-border-radius: 5px; background: #F8D030; border: 3px solid #A1871F" width="33%"> <a class="image" href="/wiki/File:Ash_Pikachu.png"><img alt="Ash Pikachu.png" height="141" src="//cdn.bulbagarden.net/upload/thumb/4/49/Ash_Pikachu.png/250px-Ash_Pikachu.png" srcset="//cdn.bulbagarden.net/upload/thumb/4/49/Ash_Pikachu.png/375px-Ash_Pikachu.png 1.5x, //cdn.bulbagarden.net/upload/thumb/4/49/Ash_Pikachu.png/500px-Ash_Pikachu.png 2x" width="250"/></a><br/><b><a href="/wiki/Ash%27s_Pikachu" title="Ash's Pikachu"><span style="color:#000;">Pikachu ↔ Gigantamax Pikachu</span></a></b> ♂
</td>
<td style="text-align:center; border-radius: 5px; -moz-border-radius: 5px; -webkit-border-radius: 5px; -khtml-border-radius: 5px; -icab-border-radius: 5px; -o-border-radius: 5px; background: #7038F8; border: 3px solid #6D5E9C" w

In [4]:
data = []
for table in tables:
    row = table.find_all('a')
    data.extend(row)

data[:10]

[<a class="image" href="/wiki/File:Ash_Pikachu.png"><img alt="Ash Pikachu.png" height="141" src="//cdn.bulbagarden.net/upload/thumb/4/49/Ash_Pikachu.png/250px-Ash_Pikachu.png" srcset="//cdn.bulbagarden.net/upload/thumb/4/49/Ash_Pikachu.png/375px-Ash_Pikachu.png 1.5x, //cdn.bulbagarden.net/upload/thumb/4/49/Ash_Pikachu.png/500px-Ash_Pikachu.png 2x" width="250"/></a>,
 <a href="/wiki/Ash%27s_Pikachu" title="Ash's Pikachu"><span style="color:#000;">Pikachu ↔ Gigantamax Pikachu</span></a>,
 <a class="image" href="/wiki/File:Ash_Dragonite.png"><img alt="Ash Dragonite.png" height="141" src="//cdn.bulbagarden.net/upload/thumb/9/91/Ash_Dragonite.png/250px-Ash_Dragonite.png" srcset="//cdn.bulbagarden.net/upload/thumb/9/91/Ash_Dragonite.png/375px-Ash_Dragonite.png 1.5x, //cdn.bulbagarden.net/upload/thumb/9/91/Ash_Dragonite.png/500px-Ash_Dragonite.png 2x" width="250"/></a>,
 <a href="/wiki/Ash%27s_Dragonite" title="Ash's Dragonite"><span style="color:#000;">Dragonite</span></a>,
 <a class="image"

In [5]:
raw_pokemon = []
for i in range(len(data)):
    if 'a href' in str(data[i]):
        raw_pokemon.append(data[i].text)
    
raw_pokemon[:20]

['Pikachu ↔ Gigantamax Pikachu',
 'Dragonite',
 'Gengar',
 'Egg →\xa0Riolu',
 "Farfetch'd",
 'Rotom Phone',
 'Bulbasaur',
 'Charmander →\xa0Charmeleon →\xa0Charizard',
 'Krabby →\xa0Kingler',
 'Muk',
 'Tauros (×30)',
 'Snorlax',
 'Heracross',
 'Chikorita →\xa0Bayleef',
 'Cyndaquil →\xa0Quilava',
 'Totodile',
 'Noctowl',
 '',
 'Egg →\xa0Phanpy →\xa0Donphan',
 'Taillow →\xa0Swellow']

In [6]:
modified_pokemon = []
for pkmn in raw_pokemon:
    pkmns = pkmn.replace('→\xa0', '→').split('→')
    modified_pokemon.extend(pkmns)

pokemon = []
for i in range(len(modified_pokemon)):
    new_pkmn = modified_pokemon[i].strip().split(' ')[0]
    if new_pkmn not in pokemon:
        pokemon.append(new_pkmn)

pokemon.remove('Egg')
pokemon.remove('')
pokemon = [pkmn if pkmn!= "Mimey" else "Mr-Mime" for pkmn in pokemon]
pokemon[:20]

['Pikachu',
 'Dragonite',
 'Gengar',
 'Riolu',
 "Farfetch'd",
 'Rotom',
 'Bulbasaur',
 'Charmander',
 'Charmeleon',
 'Charizard',
 'Krabby',
 'Kingler',
 'Muk',
 'Tauros',
 'Snorlax',
 'Heracross',
 'Chikorita',
 'Bayleef',
 'Cyndaquil',
 'Quilava']

In [7]:
pokemon_url = {}
for pkmn in pokemon:
    pokemon_url[pkmn] = ('https://pokemondb.net/pokedex/' + pkmn)

pokemon_url['Pikachu']

'https://pokemondb.net/pokedex/Pikachu'

In [8]:
raw_pokemon_data = {}
for pkmn in pokemon_url:
    try:
        re = requests.get(pokemon_url[pkmn])
        re.raise_for_status()
        raw_pokemon_data[pkmn] = (bs(re.text, 'html.parser'))
    except:
        pass
    
len(raw_pokemon_data)

94

In [9]:
raw_pokemon_stats = {}
headers = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
for pkmn in raw_pokemon_data:
    raw_pokemon_tables = raw_pokemon_data[pkmn].find_all('table')
    for table in raw_pokemon_tables:
        correct_table = True
        for header in headers:
            if header not in str(table):
                correct_table = False
        if correct_table and pkmn not in raw_pokemon_stats:
            raw_pokemon_stats[pkmn] = table
        
raw_pokemon_stats['Pikachu']

<table class="vitals-table">
<tbody>
<tr>
<th>HP</th>
<td class="cell-num">35</td>
<td class="cell-barchart">
<div class="barchart-bar barchart-rank-2" style="width:19.44%;"></div>
</td>
<td class="cell-num">180</td>
<td class="cell-num">274</td>
</tr>
<tr>
<th>Attack</th>
<td class="cell-num">55</td>
<td class="cell-barchart">
<div class="barchart-bar barchart-rank-2" style="width:30.56%;"></div>
</td>
<td class="cell-num">103</td>
<td class="cell-num">229</td>
</tr>
<tr>
<th>Defense</th>
<td class="cell-num">40</td>
<td class="cell-barchart">
<div class="barchart-bar barchart-rank-2" style="width:22.22%;"></div>
</td>
<td class="cell-num">76</td>
<td class="cell-num">196</td>
</tr>
<tr>
<th>Sp. Atk</th>
<td class="cell-num">50</td>
<td class="cell-barchart">
<div class="barchart-bar barchart-rank-2" style="width:27.78%;"></div>
</td>
<td class="cell-num">94</td>
<td class="cell-num">218</td>
</tr>
<tr>
<th>Sp. Def</th>
<td class="cell-num">50</td>
<td class="cell-barchart">
<div clas

In [10]:
regions = {1: "Kanto", 2: "Johto", 3: "Hoenn", 4: "Sinnoh", 5: "Unova", 6: "Kalos", 7: "Alola"}

pokemon_stats = []
for pkmn in raw_pokemon_stats:
    stats = {}
    raw_stats = raw_pokemon_stats[pkmn].find_all('tr')
    stats['Name'] = pkmn
    generation_text = raw_pokemon_data[pkmn].find('p').text
    generation_idx = generation_text.find('Generation')
    generation = int(generation_text[generation_idx + len('Generation ')])
    stats["Region"] = regions[generation]
    pkmn_types = raw_pokemon_data[pkmn].find('table').find_all('td')[1].find_all('a')
    stats['Type 1'] = pkmn_types[0].text
    if len(pkmn_types) > 1:
        stats['Type 2'] = pkmn_types[1].text
    else:
        stats['Type 2'] = 'None'
    for stat in raw_stats:
        stat_name = stat.find('th').get_text()
        stat_num = stat.find('td').get_text()
        if stat_name in headers:
            stats[stat_name] = stat_num
    pokemon_stats.append(stats)

pokemon_stats[:3]

[{'Name': 'Pikachu',
  'Region': 'Kanto',
  'Type 1': 'Electric',
  'Type 2': 'None',
  'HP': '35',
  'Attack': '55',
  'Defense': '40',
  'Sp. Atk': '50',
  'Sp. Def': '50',
  'Speed': '90'},
 {'Name': 'Dragonite',
  'Region': 'Kanto',
  'Type 1': 'Dragon',
  'Type 2': 'Flying',
  'HP': '91',
  'Attack': '134',
  'Defense': '95',
  'Sp. Atk': '100',
  'Sp. Def': '100',
  'Speed': '80'},
 {'Name': 'Gengar',
  'Region': 'Kanto',
  'Type 1': 'Ghost',
  'Type 2': 'Poison',
  'HP': '60',
  'Attack': '65',
  'Defense': '60',
  'Sp. Atk': '130',
  'Sp. Def': '75',
  'Speed': '110'}]

In [11]:
df = pd.DataFrame(pokemon_stats)
df.to_csv('pokemon_stats.csv')